In [18]:
import pandas as pd
import numpy as np
import os
import time
#from math import radians, cos, sin, asin, sqrt

In [19]:
cell_file = r'C:\Work\NPO Team\01 PTA Project\99 Working\07 Cell Info\Consolidated Cell Info 20230512.xlsx'

In [22]:
# Load the data into a pandas DataFrame
df = pd.read_excel(cell_file,sheet_name='cell_info',skiprows=0)

# Create a new column to store the closest "CellName" and its distance
df['Closest CellName'] = ''
df['Closest Distance'] = ''
df['Closest RF Cell Name'] = ''

for i, row in df.iterrows():
    # Filter the DataFrame to rows where "new PCI" value is the same and "Bandwidth" is the same and "CellName" is not the same
    filtered = df[(df['new PCI'] == row['new PCI']) 
                  & (df['Bandwidth (MHz)'] == row['Bandwidth (MHz)']) 
                  & (df['RF Cell Name'] != row['RF Cell Name'])
                 ]
    closest_cellnaming = ''
    closest_rfname = ''
    closest_distance = float('inf')
    
    # Calculate the distance between the current row and the filtered rows
    for j, f_row in filtered.iterrows():
        #lat1, lon1 = radians(row['Lat']), radians(row['Long'])
        #lat2, lon2 = radians(f_row['Lat']), radians(f_row['Long'])

        #a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        #c = 2 * asin(sqrt(a))
        
        
        lat1, lon1, lat2, lon2 = map(np.radians, [row['Lat'], row['Long'], f_row['Lat'], f_row['Long']])       
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        c = 2 * np.arcsin(np.sqrt(a))        
        # Multiply by the radius of the earth in kilometers to get the distance in kilometers
        distance = 6371 * c
        
        # Check if the distance is the closest so far
        if distance < closest_distance:
            closest_distance = distance
            closest_cellnaming = f_row['Cell Naming Convention']
            closest_rfname = f_row['RF Cell Name']
        
    # Update the Closest CellName and Closest Distance columns in the current row
    df.at[i, 'Closest CellName'] = closest_cellnaming
    df.at[i, 'Closest Distance'] = closest_distance
    df.at[i, 'Closest RF Cell Name'] = closest_rfname

# Save the DataFrame to a new Excel file
df.to_excel('closest_pair.xlsx', index=False)
